In [ ]:
# Step 1: Getting Data
import pandas as pd
import pandas_datareader.data as pdr
import datetime

tickers = ["AAPL","MSFT","FB","GOOG","TSLA"]

data = {}
attempt = 0
completed = []

start_date = datetime.date.today() - datetime.timedelta(3650)
end_date = datetime.date.today()

while attempt <= 1:
    tickers = [i for i in tickers if i not in completed]
    for ticker in tickers:
        try:
            temp_data = pdr.get_data_yahoo(ticker,start_date,end_date)
            data[ticker] = temp_data
            completed.append(ticker)
        except:
            print(f"{ticker} : Failed to fetch data")
            continue
    attempt = attempt + 1

# Step 2: Adding Indicators and Holders
import copy
import talib

data_copy = copy.deepcopy(data)
tickers = ["AAPL","MSFT","FB","GOOG","TSLA"]
signal = {}

for ticker in tickers:
    data_copy[ticker]["MA20"] = talib.MA(data_copy[ticker]["Adj Close"],timeperiod = 20)
    data_copy[ticker]["MA50"] = talib.MA(data_copy[ticker]["Adj Close"],timeperiod = 50)
    data_copy[ticker]["Return"] = 0
    data_copy[ticker].dropna(inplace = True)
    signal[ticker] = ""
    
# Step 3: Generating Signals and Orders

for ticker in tickers:
    for i in range(len(data_copy[ticker])):
        if signal[ticker] == "":
            if (data_copy[ticker]["MA20"][i] > data_copy[ticker]["MA50"][i]) and abs(data_copy[ticker]["MA20"][i-1] - data_copy[ticker]["MA50"][i-1]) < 10:
                signal[ticker] = "Buy"
            if (data_copy[ticker]["MA20"][i] < data_copy[ticker]["MA50"][i]) and abs(data_copy[ticker]["MA20"][i-1] - data_copy[ticker]["MA50"][i-1]) < 10:
                signal[ticker] = "Sell"
            data_copy[ticker].iloc[i,-1] = 0
        elif signal[ticker] == "Buy":
            if abs(data_copy[ticker]["MA20"][i] - data_copy[ticker]["MA50"][i]) < 10:
                signal[ticker] = ""
                data_copy[ticker].iloc[i,-1] = (data_copy[ticker]["Adj Close"][i] - data_copy[ticker]["Adj Close"][i-1]) / data_copy[ticker]["Adj Close"][i-1]
            elif (data_copy[ticker]["MA20"][i] < data_copy[ticker]["MA50"][i]) and abs(data_copy[ticker]["MA20"][i-1] - data_copy[ticker]["MA50"][i-1]) < 10:
                signal[ticker] = "Sell"
                data_copy[ticker].iloc[i,-1] = (data_copy[ticker]["Adj Close"][i] - data_copy[ticker]["Adj Close"][i-1]) / data_copy[ticker]["Adj Close"][i-1]
            else:
                data_copy[ticker].iloc[i,-1] = (data_copy[ticker]["Adj Close"][i] - data_copy[ticker]["Adj Close"][i-1]) / data_copy[ticker]["Adj Close"][i-1]
        elif signal[ticker] == "Sell":
            if abs(data_copy[ticker]["MA20"][i] - data_copy[ticker]["MA50"][i]) < 10:
                signal[ticker] = ""
                data_copy[ticker].iloc[i,-1] = - (data_copy[ticker]["Adj Close"][i] - data_copy[ticker]["Adj Close"][i-1]) / data_copy[ticker]["Adj Close"][i-1]
            elif (data_copy[ticker]["MA20"][i] > data_copy[ticker]["MA50"][i]) and abs(data_copy[ticker]["MA20"][i-1] - data_copy[ticker]["MA50"][i-1]) < 10:
                signal[ticker] = "Buy"
                data_copy[ticker].iloc[i,-1] = - (data_copy[ticker]["Adj Close"][i] - data_copy[ticker]["Adj Close"][i-1]) / data_copy[ticker]["Adj Close"][i-1]
            else:
                data_copy[ticker].iloc[i,-1] = - (data_copy[ticker]["Adj Close"][i] - data_copy[ticker]["Adj Close"][i-1]) / data_copy[ticker]["Adj Close"][i-1]
    
# Step 4: Calculate KPI
import numpy as np

def CAGR(DF):
    "function to calculate the Cumulative Annual Growth Rate of a trading strategy"
    df = DF.copy()
    df["Cum_Return"] = (1 + df["Return"]).cumprod()
    n = len(df)/(252)
    CAGR = (df["Cum_Return"].tolist()[-1])**(1/10) - 1
    return CAGR

def volatility(DF):
    "function to calculate annualized volatility of a trading strategy"
    df = DF.copy()
    vol = df["Return"].std() * np.sqrt(252*78)
    return vol

def sharpe(DF,rf):
    "function to calculate sharpe ratio ; rf is the risk free rate"
    df = DF.copy()
    sr = (CAGR(df) - rf)/volatility(df)
    return sr
    

def max_dd(DF):
    "function to calculate max drawdown"
    df = DF.copy()
    df["Cum_Return"] = (1 + df["Return"]).cumprod()
    df["Cum_Roll_Max"] = df["Cum_Return"].cummax()
    df["DD"] = df["Cum_Roll_Max"] - df["Cum_Return"]
    df["DD%"] = df["DD"]/df["Cum_Roll_Max"]
    max_dd = df["DD%"].max()
    return max_dd

df = pd.DataFrame()
for ticker in tickers:
    df[ticker] = data_copy[ticker]["Return"]
df["Return"] = df.mean(axis = 1)
print(CAGR(df))
print(volatility(df))
print(sharpe(df,0))
print(max_dd(df))

# Visualising Return
(df["Return"] + 1).cumprod().plot()